In [1]:
from src.data_examples.ex2_data_loader_1m import ExampleDataLoader

import pandas as pd
import numpy as np
from src.emulator import UnitProcess, TaskWrapper

pd.options.mode.chained_assignment = None

In [2]:
data_loader = ExampleDataLoader()
data_loader.download().load().remap()

using cached file cache\datasets\movielens\ml-1m.zip
extracting zip file content:
file already exists: data\ex2_1m\ml-1m
file already exists: data\ex2_1m\ml-1m\movies.dat
file already exists: data\ex2_1m\ml-1m\ratings.dat
file already exists: data\ex2_1m\ml-1m\README
file already exists: data\ex2_1m\ml-1m\users.dat


c:\Users\62896\Documents\GitHub\FederatedLearning\src\data_examples\ex2_data_loader_1m.py:152: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(
c:\Users\62896\Documents\GitHub\FederatedLearning\src\data_examples\ex2_data_loader_1m.py:152: ParserWarning: Both a converter and dtype were specified for column user_id - only the converter will be used.
  df = pd.read_csv(
c:\Users\62896\Documents\GitHub\FederatedLearning\src\data_examples\ex2_data_loader_1m.py:152: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(
c:\Users\62896\Documents\GitHub\FederatedLearning\src\data_example

In [3]:
# fixed size units

movie_size = 10e6

num_cl = 100
num_ue = data_loader.df_users['user_id'].nunique()

df_movies = data_loader.df_movies
df_movies['sizeof'] = movie_size
df_movies['movie_id'] = df_movies['movie_id'].astype('int')
df_movies.set_index('movie_id', drop=False, inplace=True)

In [4]:
df_movies

,movie_id,title,genres,sizeof
movie_id,,,,
0,0,Toy Story (1995),Animation|Children's|Comedy,10000000.0
1,1,Jumanji (1995),Adventure|Children's|Fantasy,10000000.0
2,2,Grumpier Old Men (1995),Comedy|Romance,10000000.0
3,3,Waiting to Exhale (1995),Comedy|Drama,10000000.0
4,4,Father of the Bride Part II (1995),Comedy,10000000.0
...,...,...,...,...
3947,3947,Meet the Parents (2000),Comedy,10000000.0
3948,3948,Requiem for a Dream (2000),Drama,10000000.0
3949,3949,Tigerland (2000),Drama,10000000.0


In [5]:
def handle_cache_overflow(tw, host, local_vars):
  host.run_task('handle_used_storage_calc')
  if local_vars.get('used') > local_vars.get('alloc'):
    host.run_task('handle_clean_cache')

def handle_used_storage_calc(tw, host, local_vars):
  disk = local_vars.get('disk')
  local_vars['used'] = sum([i['sizeof'] for i in disk])

def handle_clean_cache_lifo(tw, host, local_vars):
  local_vars.get('disk').pop(0)
  host.run_task('handle_used_storage_calc')

def handle_clean_cache_fifo(tw, host, local_vars):
  local_vars.get('disk').pop(-1)
  host.run_task('handle_used_storage_calc')

def handle_request(tw, host, local_vars, server_name, movie_id):
  client = None
  for child in host.children:
    if child.name == server_name:
      client = child
      break

  else:
    # no child is found
    return

  found_flag = False
  content = None

  # check if current requested content is in cached local variables
  for cached_content in local_vars['disk']:
    if cached_content['movie_id'] == movie_id:
      content = cached_content
      found_flag = True
      break

  # check if sibling nodes have requested content for lesser request penalty


  # else fetch from db directly with larger penalty
  if not found_flag:
    db = local_vars.get('movie_db')
    content = db.loc[movie_id]

  # check if content already in disk, else put item in the front of ordered list
  disk_movie_ids = [i['movie_id'] for i in local_vars['disk']]
  if content['movie_id'] in disk_movie_ids:
    local_vars['disk'].pop(disk_movie_ids.index(int(content.movie_id)))
  local_vars['disk'].append(content)

  # send content to client
  client.local_vars['content_buf'] = content
  # call clean cache event
  host.run_task('handle_cache_overflow')

def handle_send_content_request(tw, host, local_vars, movie_id):
  host.parent.run_task('handle_request', server_name=host.name, movie_id=movie_id)

In [6]:
bs = UnitProcess('BaseServer')
bs.local_vars['movie_db'] = df_movies
bs.local_vars.update({
  'alloc': movie_size * 1000,
  'used': 0,
  'disk': list(),
  'content_buf': None,
})

bs.create_task('handle_cache_overflow').set_task(handle_cache_overflow)
bs.create_task('handle_used_storage_calc').set_task(handle_used_storage_calc)
bs.create_task('handle_clean_cache').set_task(handle_clean_cache_lifo)
bs.create_task('handle_request').set_task(handle_request)

In [7]:
for i in range(num_cl):
  cl = bs.spawn_child(f'ClientServer [{i:02}]')
  cl.local_vars.update({
    'alloc': movie_size * 10,
    'used': 0,
    'disk': list(),
    'content_buf': None,
  })

  cl.create_task('handle_cache_overflow').set_task(handle_cache_overflow)
  cl.create_task('handle_used_storage_calc').set_task(handle_used_storage_calc)
  cl.create_task('handle_clean_cache').set_task(handle_clean_cache_lifo)
  cl.create_task('handle_request').set_task(handle_request)
  cl.create_task('handle_send_content_request').set_task(handle_send_content_request)


In [8]:
bs.run_task('handle_request', server_name=bs.children[0].name, movie_id=1234)
bs.children[0].local_vars['content_buf']

movie_id                       1234
title       Harold and Maude (1971)
genres                       Comedy
sizeof                   10000000.0
Name: 1234, dtype: object

In [9]:
cl = bs.children[1]
cl.run_task('handle_send_content_request', movie_id=420)
cl.local_vars['content_buf']

movie_id                     420
title        Black Beauty (1994)
genres      Adventure|Children's
sizeof                10000000.0
Name: 420, dtype: object